## The Dataset
The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification)
# Loading the Model


In [8]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape = (224,224,3),
    include_top=False
)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# Freezing the Model

In [9]:
base_model.trainable = False

# Adding new layers

In [17]:
inputs = keras.Input(shape=(224,224,3))
# Separately from setting trainable on the model, we set training to False
x = base_model(inputs, training = False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

model = keras.Model(inputs,outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_2   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


# Compiling the Model

In [18]:
model.compile(loss=keras.losses.categorical_crossentropy, metrics =['accuracy'])

# Augmenting the data

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(
    samplewise_center = 0,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range =0.1,
    height_shift_range =0.1,
    horizontal_flip = True,
    vertical_flip = True,
)
datagen_valid = ImageDataGenerator(samplewise_center = True)

# Load the Model

In [22]:
# Load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    'dataset/train/',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
)
# Load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    'dataset/test/',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
)

Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


# Train the Model

In [24]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size*0.1,
          validation_steps=valid_it.samples/valid_it.batch_size*0.1,
          epochs=10)

Epoch 1/10
34/34 [==============================] - 209s 6s/step - loss: 2.7039 - accuracy: 0.4455 - val_loss: 1.7285 - val_accuracy: 0.5764
Epoch 2/10
34/34 [==============================] - 196s 6s/step - loss: 1.1061 - accuracy: 0.6718 - val_loss: 1.0915 - val_accuracy: 0.6632
Epoch 3/10
34/34 [==============================] - 199s 6s/step - loss: 0.7131 - accuracy: 0.7866 - val_loss: 0.6060 - val_accuracy: 0.8125
Epoch 4/10
34/34 [==============================] - 227s 7s/step - loss: 0.4842 - accuracy: 0.8482 - val_loss: 0.5623 - val_accuracy: 0.8264
Epoch 5/10
34/34 [==============================] - 234s 7s/step - loss: 0.4147 - accuracy: 0.8696 - val_loss: 0.4421 - val_accuracy: 0.8715
Epoch 6/10
34/34 [==============================] - 242s 7s/step - loss: 0.3509 - accuracy: 0.8873 - val_loss: 0.4791 - val_accuracy: 0.8646
Epoch 7/10
34/34 [==============================] - 233s 7s/step - loss: 0.2798 - accuracy: 0.9008 - val_loss: 0.4448 - val_accuracy: 0.8750
Epoch 8/10
34

# Unfreeze the Model for Fine-Tuning

In [25]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = keras.losses.categorical_crossentropy , metrics = ['accuracy'])

In [28]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size*0.1,
          validation_steps=valid_it.samples/valid_it.batch_size*0.1,
          epochs=10
)

Epoch 1/10
34/34 [==============================] - 930s 26s/step - loss: 0.3110 - accuracy: 0.9045 - val_loss: 0.2096 - val_accuracy: 0.9236
Epoch 2/10
34/34 [==============================] - 889s 26s/step - loss: 0.1563 - accuracy: 0.9563 - val_loss: 0.3943 - val_accuracy: 0.8576
Epoch 3/10
34/34 [==============================] - 871s 26s/step - loss: 0.1006 - accuracy: 0.9705 - val_loss: 0.0887 - val_accuracy: 0.9583
Epoch 4/10
34/34 [==============================] - 860s 25s/step - loss: 0.0950 - accuracy: 0.9732 - val_loss: 0.1670 - val_accuracy: 0.9479
Epoch 5/10
34/34 [==============================] - 878s 26s/step - loss: 0.0459 - accuracy: 0.9830 - val_loss: 0.1173 - val_accuracy: 0.9583
Epoch 6/10
34/34 [==============================] - 886s 26s/step - loss: 0.0711 - accuracy: 0.9741 - val_loss: 0.1168 - val_accuracy: 0.9653
Epoch 7/10
34/34 [==============================] - 904s 27s/step - loss: 0.0574 - accuracy: 0.9821 - val_loss: 0.0730 - val_accuracy: 0.9757
Epoch 

# Evaluate the Model

In [29]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

84/84 [==============================] - 454s 5s/step - loss: 0.0689 - accuracy: 0.9770


[0.06890735030174255, 0.9770200252532959]